# API Data export for Griffin 
## 1. Import 

In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import tweepy
#import dotenv
import os
import yaml
import datetime
from pprint import pprint
import time
import re

In [2]:
#Import Tweets
#all_user = pd.read_csv('users.csv', lineterminator='\n')
#all_tweets = pd.read_csv('tweets.csv', lineterminator='\n')
all_user= pd.read_pickle("./users.pkl")
all_tweets = pd.read_pickle("./tweets.pkl")


In [3]:
all_tweets.head()

,author_id,conversation_id,created_at,entities,geo,id,text,withheld,retweet_count,reply_count,like_count,quote_count,media_keys,username
0,1380145054644379652,1539504187083456512,2022-06-22 07:02:55+00:00,"{'annotations': [{'start': 180, 'end': 193, 'p...",NaN,1539504187083456512,ClimateCrisis ClimateAction GlobalWarming Frid...,NaN,2,0,1,0,NaN,EarthManat
221,1380145054644379652,1539308556096184321,2022-06-21 18:05:33+00:00,"{'annotations': [{'start': 166, 'end': 179, 'p...",NaN,1539308556096184321,ExtinctionRebellion ClimateAction ClimateCrisi...,NaN,3,0,1,0,NaN,EarthManat
442,1380145054644379652,1539268828781334531,2022-06-21 15:27:41+00:00,"{'annotations': [{'start': 179, 'end': 192, 'p...",NaN,1539268828781334531,Renewables Greenwashing ClimateStrike Exti...,NaN,1,0,1,0,NaN,EarthManat
663,1380145054644379652,1539119110105321475,2022-06-21 05:32:45+00:00,"{'annotations': [{'start': 166, 'end': 179, 'p...",NaN,1539119110105321475,ClimateAction ClimateCrisis GlobalWarming Fri...,NaN,2,0,2,0,NaN,EarthManat
884,1380145054644379652,1538959548400160769,2022-06-20 18:58:43+00:00,"{'annotations': [{'start': 180, 'end': 193, 'p...",NaN,1538959548400160769,ClimateCrisis ClimateAction GlobalWarming Frid...,NaN,3,0,2,0,NaN,EarthManat


## 2. User/ Nodes Mapping

In [4]:
nodes = all_user.drop(['created_at', 'description','location', 'pinned_tweet_id', 'profile_image_url', 'protected', 'url', 'verified' ], axis = 1)
nodes = nodes.rename(columns = {'following_count': 'friends_count', 'tweet_count' : 'statuses_count'}, errors="raise")
nodes

,author_id,name,username,withheld,followers_count,friends_count,statuses_count,listed_count
0,1380145054644379652,man is human,EarthManat,NaN,1234,20,14631,1
1,1425101738957283339,sacccay,sacccay,NaN,272,282,43294,0
2,2817792555,WIE ICH FM,wieichfm,NaN,211,1529,5861,40
3,1519629408256024578,Le Dernier Roi de Sarrebruck,betonmicha_,NaN,0,31,4,0
4,1421341587649540097,Green Actions Senegal 🌳,Greenactions221,NaN,1288,627,1191,17
...,...,...,...,...,...,...,...,...
107158,1026803466142969856,End Climate Silence,EndClimtSilence,NaN,12905,155,3256,337
107159,258039831,Jim Scheer,jimmerz50,NaN,2096,2942,7225,24
107160,495088988,SEAI Schools,SEAI_Schools,NaN,2512,616,2381,23
107161,1265534043787010048,wedonthavetime0,wedonthavetime0,NaN,49,11,16,3


## 3. Tweet/ Edges Mapping

In [5]:
# exclude all Tweets whitout mentions
all_tweets_entities_null = all_tweets 
all_tweets_entities_null=all_tweets_entities_null[all_tweets_entities_null['entities'].isna()]
all_tweets_entities_null

,author_id,conversation_id,created_at,entities,geo,id,text,withheld,retweet_count,reply_count,like_count,quote_count,media_keys,username
262548,1380145054644379652,1430030007389327361,2021-08-24 04:51:37+00:00,NaN,NaN,1430030007389327361,1\nYou know the story of Oil companies\nWho sa...,NaN,4,1,6,1,NaN,EarthManat
43656182,1177083033385676800,1242471368878260226,2020-03-24 15:20:33+00:00,NaN,NaN,1242471371546001408,Die anderen FridaysForFuture YouTube Accounts ...,NaN,0,0,3,0,NaN,FFFDe_YouTube
44723247,3208594605,1417087749237231620,2021-07-19 11:43:42+00:00,NaN,NaN,1417087749237231620,Weil hier speziell von Klimaleugnern die Mär v...,NaN,12,3,68,1,NaN,JmxMertens
50497140,1535015724359507969,1538917629905326080,2022-06-20 16:12:09+00:00,NaN,NaN,1538917629905326080,Wir brauchen jetzt einen revolutionären Vorsch...,NaN,1,0,0,0,NaN,KadseUschi
50714065,1086267723196362753,1505957355204321294,2022-03-21 17:19:48+00:00,NaN,NaN,1505957358652076041,Und nach der Laufdemo steht noch We Say So auf...,NaN,0,1,4,0,NaN,fff_ulm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138290117,780885738963496960,1096472978051739649,2019-02-15 18:15:04+00:00,NaN,NaN,1096472978051739649,Go kids go its your futureschoolstrike4climate,NaN,0,0,0,0,NaN,PaulDav86450729
138294391,445669453,1096394929566351360,2019-02-15 13:04:55+00:00,NaN,NaN,1096394929566351360,thousands upon thousands of pedophiles followi...,NaN,0,0,0,0,NaN,marenzi_
138294539,1096352842724659200,1096392366129328129,2019-02-15 12:54:44+00:00,NaN,NaN,1096392366129328129,Good luckschoolstrike4climate,NaN,0,0,0,0,NaN,AndrewS23770328
138296860,264322898,1096317088191397888,2019-02-15 07:55:37+00:00,NaN,NaN,1096317088191397888,"Great idea, perhaps they can use their time to...",NaN,0,0,0,0,NaN,ofaroming


In [6]:
# Save all Tweet with mention
all_tweets_mention = all_tweets
all_tweets_mention.dropna(subset=['entities'], inplace = True)
all_tweets_mention

,author_id,conversation_id,created_at,entities,geo,id,text,withheld,retweet_count,reply_count,like_count,quote_count,media_keys,username
0,1380145054644379652,1539504187083456512,2022-06-22 07:02:55+00:00,"{'annotations': [{'start': 180, 'end': 193, 'p...",NaN,1539504187083456512,ClimateCrisis ClimateAction GlobalWarming Frid...,NaN,2,0,1,0,NaN,EarthManat
221,1380145054644379652,1539308556096184321,2022-06-21 18:05:33+00:00,"{'annotations': [{'start': 166, 'end': 179, 'p...",NaN,1539308556096184321,ExtinctionRebellion ClimateAction ClimateCrisi...,NaN,3,0,1,0,NaN,EarthManat
442,1380145054644379652,1539268828781334531,2022-06-21 15:27:41+00:00,"{'annotations': [{'start': 179, 'end': 192, 'p...",NaN,1539268828781334531,Renewables Greenwashing ClimateStrike Exti...,NaN,1,0,1,0,NaN,EarthManat
663,1380145054644379652,1539119110105321475,2022-06-21 05:32:45+00:00,"{'annotations': [{'start': 166, 'end': 179, 'p...",NaN,1539119110105321475,ClimateAction ClimateCrisis GlobalWarming Fri...,NaN,2,0,2,0,NaN,EarthManat
884,1380145054644379652,1538959548400160769,2022-06-20 18:58:43+00:00,"{'annotations': [{'start': 180, 'end': 193, 'p...",NaN,1538959548400160769,ClimateCrisis ClimateAction GlobalWarming Frid...,NaN,3,0,2,0,NaN,EarthManat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138299235,3236603954,1056483267099197440,2018-10-28 09:50:14+00:00,"{'urls': [{'start': 14, 'end': 37, 'url': 'htt...",NaN,1056483267099197440,School Strike https:tconXSNZ5M5Ct,NaN,0,0,0,0,NaN,MAalburywodonga
138299236,483802065,1056283608581836800,2018-10-27 22:18:29+00:00,"{'hashtags': [{'start': 63, 'end': 83, 'tag': ...",NaN,1056309182381547520,@gruenradikal @Ende__Gelaende Solidarity from ...,NaN,2,0,11,0,3_1056309178405392385,earth1stLinda
138299237,483802065,1055913441997791233,2018-10-27 09:18:54+00:00,"{'hashtags': [{'start': 36, 'end': 47, 'tag': ...",NaN,1056112996257157120,@GretaThunberg Standing with you in solidarity...,NaN,0,0,2,0,3_1056112973146611713,earth1stLinda
138299238,483802065,1055975898292219905,2018-10-27 00:14:07+00:00,"{'hashtags': [{'start': 65, 'end': 72, 'tag': ...",NaN,1055975898292219905,"I dont want 2 sound alarmists, but dont you th...",NaN,4,0,7,0,NaN,earth1stLinda


In [7]:
# Unpack Mentions in new Column 
names = []

for item in all_tweets_mention['entities']:
        if 'mentions' in item: 
                names.append(item["mentions"])
        else: names.append(np.NaN)
    
all_tweets_mention['mentions'] = names
all_tweets_mention.dropna(subset=['mentions'], inplace = True)
all_tweets_mention

,author_id,conversation_id,created_at,entities,geo,id,text,withheld,retweet_count,reply_count,like_count,quote_count,media_keys,username,mentions
241111,1380145054644379652,1413992980936110086,2021-09-05 20:30:48+00:00,"{'hashtags': [{'start': 15, 'end': 29, 'tag': ...",NaN,1434615014820876290,@extinctsymbol ClimateChange GlobalWarming Upr...,NaN,5,1,6,0,3_1434614678706135044,EarthManat,"[{'start': 0, 'end': 14, 'username': 'extincts..."
277576,1380145054644379652,1424786549716045826,2021-08-09 18:38:55+00:00,"{'hashtags': [{'start': 15, 'end': 32, 'tag': ...",NaN,1424802388356050945,@ProtectThPass FridaysForFuture LithiumForFutu...,NaN,1,0,1,0,3_1424801802206187523,EarthManat,"[{'start': 0, 'end': 14, 'username': 'ProtectT..."
282438,1380145054644379652,1423662685141536770,2021-08-06 15:42:33+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",NaN,1423670839292178624,@StephenCorrySvl FridaysForFuture push for ren...,NaN,4,1,6,0,NaN,EarthManat,"[{'start': 0, 'end': 16, 'username': 'StephenC..."
298350,1380145054644379652,1417881812811063304,2021-07-21 16:31:34+00:00,"{'hashtags': [{'start': 263, 'end': 280, 'tag'...",NaN,1417884969532264450,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,1,1,1,0,NaN,EarthManat,"[{'start': 0, 'end': 7, 'username': 'faznet', ..."
299013,1380145054644379652,1417783978925375489,2021-07-21 11:19:19+00:00,"{'hashtags': [{'start': 295, 'end': 312, 'tag'...",NaN,1417806388852576257,@RinglerSchaff @FAZ_NET @wk_juergs @Luisamneub...,NaN,0,0,2,0,NaN,EarthManat,"[{'start': 0, 'end': 14, 'username': 'RinglerS..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138299196,546420483,1059484897503395840,2018-11-05 16:37:38+00:00,"{'hashtags': [{'start': 12, 'end': 33, 'tag': ...",NaN,1059484897503395840,November is schoolstrike4climate month across ...,NaN,1,0,4,0,NaN,MDeMocker,"[{'start': 142, 'end': 152, 'username': 'youth..."
138299201,21508112,1059062751413321728,2018-11-04 12:40:11+00:00,"{'hashtags': [{'start': 128, 'end': 142, 'tag'...",NaN,1059062751413321728,"Next month, kids are going on strike from scho...",NaN,4,0,10,0,3_1059062749462978560,debcastellana,"[{'start': 147, 'end': 161, 'username': 'Clima..."
138299227,107550480,1057711079344627712,2018-10-31 19:09:07+00:00,"{'annotations': [{'start': 3, 'end': 11, 'prob...",NaN,1057711079344627712,In Australia climate change impacts are fact o...,NaN,0,0,0,0,NaN,SustainableSas,"[{'start': 231, 'end': 245, 'username': 'Strik..."
138299236,483802065,1056283608581836800,2018-10-27 22:18:29+00:00,"{'hashtags': [{'start': 63, 'end': 83, 'tag': ...",NaN,1056309182381547520,@gruenradikal @Ende__Gelaende Solidarity from ...,NaN,2,0,11,0,3_1056309178405392385,earth1stLinda,"[{'start': 0, 'end': 13, 'username': 'gruenrad..."


In [8]:
# Create single row for each mention
all_tweets_mentions_ids = all_tweets_mention
all_tweets_mentions_ids = all_tweets_mentions_ids.explode('mentions')

all_tweets_mentions_ids

,author_id,conversation_id,created_at,entities,geo,id,text,withheld,retweet_count,reply_count,like_count,quote_count,media_keys,username,mentions
241111,1380145054644379652,1413992980936110086,2021-09-05 20:30:48+00:00,"{'hashtags': [{'start': 15, 'end': 29, 'tag': ...",NaN,1434615014820876290,@extinctsymbol ClimateChange GlobalWarming Upr...,NaN,5,1,6,0,3_1434614678706135044,EarthManat,"{'start': 0, 'end': 14, 'username': 'extinctsy..."
277576,1380145054644379652,1424786549716045826,2021-08-09 18:38:55+00:00,"{'hashtags': [{'start': 15, 'end': 32, 'tag': ...",NaN,1424802388356050945,@ProtectThPass FridaysForFuture LithiumForFutu...,NaN,1,0,1,0,3_1424801802206187523,EarthManat,"{'start': 0, 'end': 14, 'username': 'ProtectTh..."
282438,1380145054644379652,1423662685141536770,2021-08-06 15:42:33+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",NaN,1423670839292178624,@StephenCorrySvl FridaysForFuture push for ren...,NaN,4,1,6,0,NaN,EarthManat,"{'start': 0, 'end': 16, 'username': 'StephenCo..."
298350,1380145054644379652,1417881812811063304,2021-07-21 16:31:34+00:00,"{'hashtags': [{'start': 263, 'end': 280, 'tag'...",NaN,1417884969532264450,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,1,1,1,0,NaN,EarthManat,"{'start': 0, 'end': 7, 'username': 'faznet', '..."
298350,1380145054644379652,1417881812811063304,2021-07-21 16:31:34+00:00,"{'hashtags': [{'start': 263, 'end': 280, 'tag'...",NaN,1417884969532264450,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,1,1,1,0,NaN,EarthManat,"{'start': 8, 'end': 18, 'username': 'wk_juergs..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138299236,483802065,1056283608581836800,2018-10-27 22:18:29+00:00,"{'hashtags': [{'start': 63, 'end': 83, 'tag': ...",NaN,1056309182381547520,@gruenradikal @Ende__Gelaende Solidarity from ...,NaN,2,0,11,0,3_1056309178405392385,earth1stLinda,"{'start': 0, 'end': 13, 'username': 'gruenradi..."
138299236,483802065,1056283608581836800,2018-10-27 22:18:29+00:00,"{'hashtags': [{'start': 63, 'end': 83, 'tag': ...",NaN,1056309182381547520,@gruenradikal @Ende__Gelaende Solidarity from ...,NaN,2,0,11,0,3_1056309178405392385,earth1stLinda,"{'start': 14, 'end': 29, 'username': 'Ende__Ge..."
138299237,483802065,1055913441997791233,2018-10-27 09:18:54+00:00,"{'hashtags': [{'start': 36, 'end': 47, 'tag': ...",NaN,1056112996257157120,@GretaThunberg Standing with you in solidarity...,NaN,0,0,2,0,3_1056112973146611713,earth1stLinda,"{'start': 0, 'end': 14, 'username': 'GretaThun..."
138299237,483802065,1055913441997791233,2018-10-27 09:18:54+00:00,"{'hashtags': [{'start': 36, 'end': 47, 'tag': ...",NaN,1056112996257157120,@GretaThunberg Standing with you in solidarity...,NaN,0,0,2,0,3_1056112973146611713,earth1stLinda,"{'start': 118, 'end': 132, 'username': 'jacind..."


In [9]:
# Save mentions username as dst_name 
usernames = []

for item in all_tweets_mentions_ids['mentions']:
    usernames.append(item['username'])
    
all_tweets_mentions_ids['dst_name'] = usernames
all_tweets_mentions_ids.head()

,author_id,conversation_id,created_at,entities,geo,id,text,withheld,retweet_count,reply_count,like_count,quote_count,media_keys,username,mentions,dst_name
241111,1380145054644379652,1413992980936110086,2021-09-05 20:30:48+00:00,"{'hashtags': [{'start': 15, 'end': 29, 'tag': ...",NaN,1434615014820876290,@extinctsymbol ClimateChange GlobalWarming Upr...,NaN,5,1,6,0,3_1434614678706135044,EarthManat,"{'start': 0, 'end': 14, 'username': 'extinctsy...",extinctsymbol
277576,1380145054644379652,1424786549716045826,2021-08-09 18:38:55+00:00,"{'hashtags': [{'start': 15, 'end': 32, 'tag': ...",NaN,1424802388356050945,@ProtectThPass FridaysForFuture LithiumForFutu...,NaN,1,0,1,0,3_1424801802206187523,EarthManat,"{'start': 0, 'end': 14, 'username': 'ProtectTh...",ProtectThPass
282438,1380145054644379652,1423662685141536770,2021-08-06 15:42:33+00:00,"{'mentions': [{'start': 0, 'end': 16, 'usernam...",NaN,1423670839292178624,@StephenCorrySvl FridaysForFuture push for ren...,NaN,4,1,6,0,NaN,EarthManat,"{'start': 0, 'end': 16, 'username': 'StephenCo...",StephenCorrySvl
298350,1380145054644379652,1417881812811063304,2021-07-21 16:31:34+00:00,"{'hashtags': [{'start': 263, 'end': 280, 'tag'...",NaN,1417884969532264450,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,1,1,1,0,NaN,EarthManat,"{'start': 0, 'end': 7, 'username': 'faznet', '...",faznet
298350,1380145054644379652,1417881812811063304,2021-07-21 16:31:34+00:00,"{'hashtags': [{'start': 263, 'end': 280, 'tag'...",NaN,1417884969532264450,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,1,1,1,0,NaN,EarthManat,"{'start': 8, 'end': 18, 'username': 'wk_juergs...",wk_juergs


In [10]:
edges_mentions = all_tweets_mentions_ids 
edges_mentions = all_tweets_mentions_ids.drop(['conversation_id', 'entities','geo', 'id', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'media_keys','mentions'], axis = 1)
edges_mentions = edges_mentions.rename(columns = {'author_id': 'src_id_str', 'username' : 'src_name'})
edges_mentions

,src_id_str,created_at,text,withheld,src_name,dst_name
241111,1380145054644379652,2021-09-05 20:30:48+00:00,@extinctsymbol ClimateChange GlobalWarming Upr...,NaN,EarthManat,extinctsymbol
277576,1380145054644379652,2021-08-09 18:38:55+00:00,@ProtectThPass FridaysForFuture LithiumForFutu...,NaN,EarthManat,ProtectThPass
282438,1380145054644379652,2021-08-06 15:42:33+00:00,@StephenCorrySvl FridaysForFuture push for ren...,NaN,EarthManat,StephenCorrySvl
298350,1380145054644379652,2021-07-21 16:31:34+00:00,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,EarthManat,faznet
298350,1380145054644379652,2021-07-21 16:31:34+00:00,@faznet @wk_juergs 22 Gleichzeitig fördern sie...,NaN,EarthManat,wk_juergs
...,...,...,...,...,...,...
138299236,483802065,2018-10-27 22:18:29+00:00,@gruenradikal @Ende__Gelaende Solidarity from ...,NaN,earth1stLinda,gruenradikal
138299236,483802065,2018-10-27 22:18:29+00:00,@gruenradikal @Ende__Gelaende Solidarity from ...,NaN,earth1stLinda,Ende__Gelaende
138299237,483802065,2018-10-27 09:18:54+00:00,@GretaThunberg Standing with you in solidarity...,NaN,earth1stLinda,GretaThunberg
138299237,483802065,2018-10-27 09:18:54+00:00,@GretaThunberg Standing with you in solidarity...,NaN,earth1stLinda,jacindaardern


In [11]:
edges_mentions_src = edges_mentions 
edges_mentions_src = edges_mentions_src.merge(nodes, how = 'inner', left_on='src_name', right_on='username')
edges_mentions_dst = edges_mentions_src.merge(nodes, how = 'inner', left_on='dst_name', right_on='username')
edges_mentions_dst = edges_mentions_dst.drop(['listed_count_y', 'statuses_count_y', 'friends_count_y', 'followers_count_y', 'author_id_x', 'username_x', 'username_y' ], axis = 1)
edges_mentions_dst = edges_mentions_dst.rename(columns = {'name_x': 'src_screen_name', 'followers_count_x' : 'src_followers_count', 'friends_count_x': 'src_friends_count', 'statuses_count_x': 'src_statuses_count', 'listed_count_x': 'src_listed_count', 'author_id_y':'dst_id_str', 'name_y': 'dst_screen_name',  })
edges_mentions_dst

MemoryError: Unable to allocate 560. GiB for an array with shape (75225656210,) and data type int64

## 3. Export Nodes and Edges 

In [ ]:
edges_mentions_dst.to_csv("edges.csv", index=False)
nodes.to_csv("nodes.csv", index=False)